# Seldon V2 Non Kubernetes Local Examples


 * Build if needed and place `seldon` binary in your path
 * Install seldon locally 

In [ ]:
!which seldon

## Model

In [ ]:
!cat ./models/sklearn-iris-gs.yaml

In [ ]:
!seldon model load -f ./models/sklearn-iris-gs.yaml

In [ ]:
!seldon model status --model-name iris -w ModelAvailable | jq .

In [ ]:
!seldon model infer --model-name iris \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

In [ ]:
!seldon model infer --model-name iris --inference-mode grpc \
   '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' | jq .

In [ ]:
!seldon model unload --model-name iris

## Experiment

In [ ]:
!cat ./experiments/sklearn1.yaml

In [ ]:
!cat ./experiments/sklearn2.yaml

In [ ]:
!seldon model load -f ./experiments/sklearn1.yaml
!seldon model load -f ./experiments/sklearn2.yaml

In [ ]:
!seldon model status --model-name iris | jq .
!seldon model status --model-name iris2 | jq .

In [ ]:
!cat ./experiments/ab-default-model.yaml 

In [ ]:
!seldon experiment start -f ./experiments/ab-default-model.yaml 

In [ ]:
!seldon experiment status -e experiment-sample -w | jq .

In [ ]:
!seldon model infer --model-name iris -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

In [ ]:
!seldon experiment stop -e experiment-sample

In [ ]:
!seldon model unload --model-name iris
!seldon model unload --model-name iris2

## Pipeline

In [1]:
!cat ./models/sklearn-iris-gs.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn


In [ ]:
!seldon model load -f ./models/sklearn-iris-gs.yaml 

In [ ]:
!seldon model status --model-name iris -w ModelAvailable | jq .

In [ ]:
!seldon model infer --model-name iris \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

In [ ]:
!cat ./pipelines/iris.yaml

In [ ]:
!seldon pipeline load -f ./pipelines/iris.yaml

In [ ]:
!seldon pipeline status -p iris -w PipelineReady | jq .

In [ ]:
!seldon pipeline infer -p iris \
   '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

In [ ]:
!seldon pipeline unload -p iris

In [ ]:
!seldon pipeline status -p iris| jq .

In [ ]:
!seldon model unload --model-name iris